In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing import image

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
folder_path = "/content/drive/MyDrive/xray_bone_fracture/"

training_path = folder_path + "train"
testing_path = folder_path + "val"


In [4]:
training_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
)

testing_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1)

In [5]:
training_generator = training_datagen.flow_from_directory(
    training_path,
    target_size = (224,224),
    batch_size = 4,
    class_mode = 'binary')
testing_generator = testing_datagen.flow_from_directory(
    testing_path,
    target_size = (224,224),
    batch_size = 4,
    shuffle=True,
    class_mode = 'binary')

Found 8863 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [15]:
base_model = tf.keras.applications.EfficientNetB3(weights='imagenet', input_shape=(224,224,3), include_top=False)

for layer in base_model.layers:
    layer.trainable=False
model = Sequential()
model.add(base_model)
#model.add(Dense(512, activation = 'relu'))
#model.add(GaussianNoise(0.25))
model.add(GlobalAveragePooling2D())
#model.add(Dense(512,activation='relu'))
#model.add(BatchNormalization())
#model.add(GaussianNoise(0.25))
#model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))
#model.add(GaussianNoise(0.25))
#model.add(Dropout(0.25))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb3 (Functional  (None, 7, 7, 1536)        10783535  
 )                                                               
                                                                 
 global_average_pooling2d_1  (None, 1536)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_3 (Dense)             (None, 1)                 1537      
                                                                 
Total params: 10785072 (41.14 MB)
Trainable params: 1537 (6.00 KB)
Non-trainable params: 10783535 (41.14 MB)
_________________________________________________________________


In [16]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy','Precision','Recall','AUC'])

In [17]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
lrp=ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=2)
filepath='best_model.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
call=[checkpoint,lrp]
history = model.fit(
    training_generator,
    epochs=20,
    validation_data=testing_generator,
    steps_per_epoch= 50,
    callbacks=call
    )

Epoch 1/20
50/50 [==============================] - ETA: 0s - loss: 0.6890 - accuracy: 0.5600 - precision: 0.5725 - recall: 0.7009 - auc: 0.5643
Epoch 1: val_accuracy improved from -inf to 0.58500, saving model to best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


50/50 [==============================] - 301s 6s/step - loss: 0.6890 - accuracy: 0.5600 - precision: 0.5725 - recall: 0.7009 - auc: 0.5643 - val_loss: 0.6613 - val_accuracy: 0.5850 - val_precision: 0.4906 - val_recall: 0.9750 - val_auc: 0.8212 - lr: 0.0010
Epoch 2/20
50/50 [==============================] - ETA: 0s - loss: 0.6265 - accuracy: 0.6550 - precision: 0.5952 - recall: 0.5882 - auc: 0.7052
Epoch 2: val_accuracy improved from 0.58500 to 0.72500, saving model to best_model.h5
50/50 [==============================] - 157s 3s/step - loss: 0.6265 - accuracy: 0.6550 - precision: 0.5952 - recall: 0.5882 - auc: 0.7052 - val_loss: 0.5746 - val_accuracy: 0.7250 - val_precision: 0.6344 - val_recall: 0.7375 - val_auc: 0.8001 - lr: 0.0010
Epoch 3/20
50/50 [==============================] - ETA: 0s - loss: 0.6023 - accuracy: 0.6650 - precision: 0.6458 - recall: 0.6526 - auc: 0.7344
Epoch 3: val_accuracy did not improve from 0.72500
50/50 [==============================] - 213s 4s/step - los

In [18]:
model.evaluate(training_generator)

2216/2216 [==============================] - 2105s 950ms/step - loss: 0.5302 - accuracy: 0.7680 - precision: 0.7372 - recall: 0.8250 - auc: 0.8483


[0.5301942229270935,
 0.7680243849754333,
 0.7372069358825684,
 0.8250057101249695,
 0.848292350769043]

In [19]:
model.evaluate(testing_generator)

150/150 [==============================] - 89s 595ms/step - loss: 0.5709 - accuracy: 0.6983 - precision: 0.5845 - recall: 0.8500 - auc: 0.7780


[0.5709335803985596,
 0.6983333230018616,
 0.5845271944999695,
 0.8500000238418579,
 0.778003454208374]